In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/till-30000/till_30000.csv
/kaggle/input/model-till-25000/config.json
/kaggle/input/model-till-25000/merges.txt
/kaggle/input/model-till-25000/vocab.json
/kaggle/input/model-till-25000/tokenizer_config.json
/kaggle/input/model-till-25000/model.safetensors
/kaggle/input/model-till-25000/special_tokens_map.json
/kaggle/input/model-till-25000/added_tokens.json
/kaggle/input/model-till-25000/generation_config.json
/kaggle/input/till-20000/till_20000.csv


In [2]:
import torch

In [3]:
df = pd.read_csv("/kaggle/input/till-30000/till_30000.csv")

In [5]:
df1 = pd.read_csv("/kaggle/input/till-20000/till_20000.csv")

In [6]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [7]:
df.head()

,title,article,summary
0,You Need to Care About Facebook Censoring an I...,Nick Ut's Pulitzer Prize-winning photograph of...,Nick Ut's Pulitzer Prize-winning photograph of...
1,Explainer: Counting the cost of the U.S.-China...,HONG KONG (Reuters) - U.S. President Donald Tr...,U.S. President Donald Trump's fresh threats of...
2,'Breaking Bad' Star Lavell Crawford Dropped 12...,Comedian and actor Lavell Crawford's not quite...,Comedian and actor Lavell Crawford's not quite...
3,"Daily Horoscope: November 09, 2016",Mars enters Aquarius today at 12:52 AM! Mars i...,Mars enters Aquarius today at 12:52 AM! Mars i...
4,"UK's May tries to calm Brexit rebels, says dea...",LONDON (Reuters) - Prime Minister Theresa May ...,LONDON (Reuters) - Prime Minister Theresa May ...


In [8]:
df['summary'] = df['summary'].str.replace(r'\s{2,}', ' ', regex=True).str.strip()

In [16]:
'''from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.add_special_tokens({"pad_token":"<pad>"})
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
model.resize_token_embeddings(len(tokenizer))'''

from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = "/kaggle/input/model-till-25000"

# # Load the model
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
# tokenizer.add_special_tokens({"pad_token":"<pad>"})
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
# model.resize_token_embeddings(len(tokenizer))

# If GPU is available, move the model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)



GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50258, bias=False)
)

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

messages = df["summary"][1]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 But the trade war has also hurt China's economy. The country's economic growth has slowed to 6.5 percent in the first quarter of this year, the lowest in 26 years.The trade war has also hurt the U.S. economy. The International Monetary Fund has cut its forecast for U.S. growth this year to 2.1 percent from 2.3 percent. The U.S. economy grew at a 2.9 percent annual rate in the first quarter of this year.The U.S. trade deficit with China has also widened. The deficit was $37.2 billion in March, the highest since September 2008.The U.S. trade deficit with China has widened to $37.2 billion in March, the highest since September 2008. The deficit was $37.1 billion in February and $36.6 billion in January.The U.S. trade deficit with China has widened to $37.2 billion in March, the highest since September 2008. The deficit was $37.1 billion in February and $36.6 billion in January.The U.S. trade deficit with China has widened to $37.2 billion in March, the highest since September 2008. The d

### **Training GPT2**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import Dataset, load_dataset

In [ ]:
df["gpt2_summary"] = df["summary"] + '\nTL;DR:' + df["title"]

In [ ]:
df1["gpt2_summary"] = df1["summary"] + '\nTL;DR:' + df1["title"]

In [ ]:
new_df = df[['gpt2_summary']].iloc[2000:4000]
eval_df = df1[['gpt2_summary']].iloc[4000:4100]

In [ ]:
new_df = Dataset.from_pandas(new_df)
eval_df = Dataset.from_pandas(eval_df)

In [ ]:
new_df

In [ ]:
eval_df

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['summary'] , max_length = 1024, padding = True, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['title'], max_length = 32, padding = True, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    
new_df_pt = new_df.map(convert_examples_to_features, batched = True)
eval_df_pt = eval_df.map(convert_examples_to_features, batched = True)

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments, Trainer
from accelerate import Accelerator, DataLoaderConfiguration

# Create DataLoaderConfiguration object
dataloader_config = DataLoaderConfiguration(
    dispatch_batches=None,  # Change None to your desired value
    split_batches=False,
    even_batches=True,
    use_seedable_sampler=True
)

# Create Accelerator object with DataLoaderConfiguration
accelerator = Accelerator(dataloader_config=dataloader_config)


# Define training arguments with gradient accumulation, dropout, and validation
trainer_args = TrainingArguments(
    output_dir='news_headline',
    num_train_epochs=10,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=15,
    gradient_accumulation_steps=16,
    learning_rate=3e-5,  # Specify the learning rate here
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    overwrite_output_dir=True,
    dataloader_num_workers=accelerator.num_processes
)


In [ ]:
# Now you can use the accelerator object in your Trainer initialization
trainer = Trainer(
    model=model,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=new_df_pt,
    eval_dataset=eval_df_pt
)


In [ ]:
train_result = trainer.train()

In [ ]:
trainer.evaluate(metric_key_prefix='test_en',
                eval_dataset=en_test_dataset)

In [ ]:
model =  model.eval()

In [17]:
sentence = output[0]['generated_text']
query = sentence + '\nTL;DR:'

In [20]:
input_ids = tokenizer(query, return_tensors='pt')

In [21]:
# Move the input to GPU if available
input_ids = input_ids.to("cuda")

# Generate 5 different outputs
num_outputs = 7
generated_outputs = []
max_new_tokens = 20
for _ in range(num_outputs):
    # Generate output using the model
    output = model.generate(
        input_ids=input_ids["input_ids"],
        attention_mask=input_ids['attention_mask'],
        length_penalty=0.8,
        min_new_tokens=7,
        max_new_tokens=max_new_tokens+_,
        num_beams=8,
        no_repeat_ngram_size=2,
        do_sample=True,
        early_stopping=True
    )
    generated_outputs.append(tokenizer.decode(output[0],skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [22]:
# Print the generated outputs
for i, output_text in enumerate(generated_outputs):
    # Find the index of "TL;DR:"
    tldr_index = output_text.find("TL;DR:")
    if tldr_index != -1:
        # Extract the text after "TL;DR:"
        output_text = output_text[tldr_index + len("TL;DR:"):]
    
    # Remove commas from the output text
    output_text = output_text.replace(",", "")

    print(f"Output {i+1}: {output_text.strip()}")  # Strip to remove leading/trailing whitespace

Output 1: China's trade with the United States is hurting its economy but the damage is far from done
Output 2: Factbox: China trade with the United States is hurting the world economy and it's getting worse
Output 3: China's trade with the United States is hurting the economy  : This article originally appeared on Rec
Output 4: China trade woes hit China economy but it's not as bad as it used to Be sure to sign up for
Output 5: China's trade with the United States is hurting the world economy but the damage is far worse than you think
Output 6: China's trade with the United States is hurting its economy but it's not as bad as it used to be
Output 7: Factbox: China trade with the United States is the biggest drag on the world economy  - The Wall Street Journal


In [23]:
df['title'][1] 

'Explainer: Counting the cost of the U.S.-China trade war so far'

In [ ]:
!pip install rouge

In [ ]:
sentence = '''
BENGALURU: Over 69% of voters in 14 Lok Sabha constituencies of coastal and southern Karnataka turned up to exercise their franchise on Friday, and the provisional voting percentage almost mirrors the figures notched up in the previous two polls: 70.4% in 2019 and 68% in 2014.
Bengaluru continued to remain on a sticky wicket with its overall subpar performance of 57%.Mysuru, for the first time, hit the ball out of the park crossing 70% against 69.5% in 2019 and 67.3% in 2014. With this round of polling, the fate of 247 candidates has been sealed, and the remaining 14 seats in central and northern Karnataka will vote on May 7.
The seemingly middling turnout in the first phase is yet commendable as voters braved the heat to keep their tryst with the voting exercise. After the final tabulation on the morrow, the overall voting percentage is expected to rise by a couple of percentage of points, helping the figure sail over the psychological 70 mark.
Of the four seats, Bangalore Rural reported the best turnout of 67.3% compared to 64.9% in 2019 and Central recor ded 52.8% compared to 54.3% in 2019, while it was 53.1% in South (53.7% in 2019). North had breached the 50% mark by 5pm and ended with 54.4%, compared to 54.8% in 2019.
All four seats had seen a dip in turnout compared to 2014, with Bangalore South recording the least.
The low turnout in the four-seat Bengaluru cluster is attributable to high temperatures and largely to continuing apathy. The issue of electoral roll inflation also continues to plague Bengaluru.
Mandya, which had seen a lot of campaigning action in the run-up to the polling day, emerged on top again, registering 81.5% turnout.
In 2019 and 2014 too, the tally had crossed 80% mark, making them the most robust voting constituency in the state.
The 14 seats continued to witness the trend of rural voters pulling up the overall voting percentage in Karnataka with Bengaluru’s low proving to be a drag. The lowest polling was recorded in Bangalore Central with just over 52%.
In a bid to beat the late forenoon and afternoon heat, voters stood in long queues early in the morning across the state.
The voter turnout was the highest between 7am and 11am with 22% and post 3pm when Karnataka saw close to 18% of the total voters casting their ballot.
Barring isolated cases of vandalism, missing votes and mismatches in the electoral roll, leading to a few voters being unable to cast their ballot, the state saw peaceful voting across the constituencies.
As the day progressed and temperatures soared past 37°C, Bengalureans failed to show much enthusiasm in exercising their franchise.
“We expected the city to perform better this time. After multiple consultations and reviews with local authorities, we had identified problematic areas and planned a targeted approach to educate voters and create awareness. Yet, the city did n ot improve on its previous tally,” said a senior official from the Chief Elec toral Office of Karnataka.
Political parties, too, were upset about the outcome and blamed stringent measures initiated by the Election Commission for the poor turnout.
A candidate from Bangalore Central said: “There are restrictions on holding community engagement events in clubs and marriage halls. We will be charged expenses if a bike or any other rally is held. Given the time constraint, it is difficult to connect with a ll voters through doorto-door campaigns alone.”
“More events would have increased awareness among the general public. Digital platforms help in reaching out to one section of society but not all,” the candidate pointed out.'''

In [ ]:
tokens = tokenizer.tokenize(sentence)

# Count the number of tokens
num_tokens = len(tokens)

print("Number of tokens:", num_tokens)


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from rouge import Rouge


# Define the generated summary and the reference summary
reference_summary = df["title"][1900]

# Initialize the ROUGE object
rouge = Rouge()

# Define the input text for GPT-2
input_text = summarize(model_bert, df["article"][1900]) + '\nTL;DR:'

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors="pt").to("cuda")

# Generate predictions
output = model.generate(input_ids=input_ids,
                        length_penalty=0.8,
                        min_new_tokens=7,
                        max_new_tokens=24,
                        num_beams=8,
                        no_repeat_ngram_size=2,
                        do_sample=True,
                        early_stopping=True,
                        temperature=0.7,
                        top_k=50,
                        top_p=0.95,
                        num_return_sequences=2)

# Decode the generated sequence
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
tldr_index = output_text.find("TL;DR:")
if tldr_index != -1:
    # Extract the text after "TL;DR:"
    output_text = output_text[tldr_index + len("TL;DR:"):]

#Remove commas from the output text
output_text = output_text.replace(",", "")

generated_text = output_text.strip()

# Calculate ROUGE for the generated and reference summaries
scores = rouge.get_scores(generated_text, reference_summary)
# Print the results
print(scores)


In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
from tqdm import tqdm

def calculate_rouge_scores(df, model, tokenizer):
    rouge_scores = []

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Calculating ROUGE scores"):
        # Define the reference summary
        reference_summary = row["title"]
        max_new_tokens = len(reference_summary)

        # Define the input text for GPT-2
        input_text = summarize(model_bert, row["article"]) + '\nTL;DR:'

        # Tokenize input text
        input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

        # Generate predictions
        output = model.generate(input_ids=input_ids["input_ids"],
                                attention_mask=input_ids["attention_mask"],
                                length_penalty=0.8,
                                min_new_tokens=7,
                                max_new_tokens=max_new_tokens,
                                num_beams=8,
                                no_repeat_ngram_size=2,
                                do_sample=True,
                                early_stopping=True,
                                temperature=0.7,
                                top_k=50,
                                top_p=0.95,
                                num_return_sequences=2)

        # Decode the generated sequence
        output_text = tokenizer.decode(output[0], skip_special_tokens=True)
        tldr_index = output_text.find("TL;DR:")
        if tldr_index != -1:
            # Extract the text after "TL;DR:"
            output_text = output_text[tldr_index + len("TL;DR:"):]

        # Remove commas from the output text
        output_text = output_text.replace(",", "")

        generated_text = output_text.strip()

        # Calculate ROUGE for the generated and reference summaries
        scores = rouge.get_scores(generated_text, reference_summary)
        rouge_scores.append(scores)

    return rouge_scores

# Example usage:
rouge_scores = calculate_rouge_scores(df[:400], model, tokenizer)


In [ ]:
# Compute average scores
rouge1_avg = sum(score[0]['rouge-1']['f'] for score in rouge_scores) / len(rouge_scores)
rouge2_avg = sum(score[0]['rouge-2']['f'] for score in rouge_scores) / len(rouge_scores)
rougeL_avg = sum(score[0]['rouge-l']['f'] for score in rouge_scores) / len(rouge_scores)

# Create a dictionary of average scores
avg_scores = {
    'rouge1_avg': rouge1_avg*100,
    'rouge2_avg': rouge2_avg*100,
    'rougeL_avg': rougeL_avg*100
}

# Display average scores in a table
df_avg_scores = pd.DataFrame.from_dict(avg_scores, orient='index', columns=['Average Score'])
print(df_avg_scores)


In [ ]:
generated_text

In [ ]:
df["gpt2_summary"][13]

In [ ]:
!pip install bert_score

In [ ]:
from bert_score import score

hypotheses = [df["title"][1900]]
# Define the input text for GPT-2
input_text = df["summary"][29] + '\nTL;DR:'

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors="pt").to("cuda")

# Generate predictions
output = model.generate(input_ids=input_ids,
                        length_penalty=0.8,
                        min_new_tokens=7,
                        max_new_tokens=30,
                        num_beams=8,
                        no_repeat_ngram_size=2,
                        do_sample=True,
                        early_stopping=True,
                        temperature=0.7,
                        top_k=50,
                        top_p=0.95,
                        num_return_sequences=1)

# Decode the generated sequence
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
tldr_index = output_text.find("TL;DR:")
if tldr_index != -1:
    # Extract the text after "TL;DR:"
    output_text = output_text[tldr_index + len("TL;DR:"):]

# Remove commas from the output text
output_text = output_text.replace(",", "")

generated_text = output_text.strip()
references = [generated_text]

lang = "en"

P, R, F1 = score(hypotheses, references, lang=lang)

In [ ]:
print("Precision: ", P[0],"\nCoverage: ",R[0], "\nF1 Score: ",F1[0])

In [ ]:
!pip install -U nltk

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
import nltk
from nltk.translate.meteor_score import meteor_score

reference = [df["title"][2900].split()]
input_text = df["summary"][2900] + '\nTL;DR:'
max_new_tokens = len(reference[0])
# Tokenize input text
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate predictions
output = model.generate(input_ids=input_ids["input_ids"],
                        attention_mask=input_ids['attention_mask'],
                        length_penalty=0.8,
                        min_new_tokens=7,
                        max_new_tokens=30,
                        num_beams=8,
                        no_repeat_ngram_size=2,
                        do_sample=True,
                        early_stopping=True)

# Decode the generated sequence
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
tldr_index = output_text.find("TL;DR:")
if tldr_index != -1:
    # Extract the text after "TL;DR:"
    output_text = output_text[tldr_index + len("TL;DR:"):]

# Remove commas from the output text
output_text = output_text.replace(",", "")

generated_text = output_text.strip()
generated = generated_text.split()
score = meteor_score(reference, generated)
print(score)

In [ ]:
generated_text

In [ ]:
import torch
from torchtext.data.metrics import bleu_score

# define the source text and reference text
source_text = ["This is an example message for summarization."]
reference_text = ["This is an example message for summarizing."]
# define the text generated by the model
generated_text = ["This is an example message for summarizing."]
# calculate the BLEU score
score = bleu_score(generated_text, reference_text)
print(f'BLEU Score: {score*100:.2f}')

In [ ]:
sentence = summarization('''Think of Ian Nepomniachtchi not as a grandmaster, but as an escape artist. Time and again at the Candidates tournament in Toronto, he’s found himself in losing positions, like he did against Praggnanandhaa in Round 5 and against Alireza Firouzja in Round 9. Time and again, he’s escaped unscathed. On Thursday, Nepomniachtchi performed his most jaw-dropping trick in Toronto yet, managing to survive from two losing positions before defeating Vidit Gujrathi after a marathon battle that lasted over five hours.
The victory sent Nepomniachtchi into sole lead in the open section of the Candidates chess tournament with seven points. Nepomniachtchi has Gukesh, the 17-year-old, first-timer from India, and 36-year-old Hikaru Nakamura, a two-time Candidates contender, hot on his heels.
Both have half a point less than Nepomniachtchi with just three more rounds left. Gukesh played out a draw against World No.2 Fabiano Caruana with white pieces while the third Indian in the open competition, Praggnanandhaa, lost to World No.3 Nakamura, a result that sees him 1.5 points off the pace.
Chess CandidatesWhen the end came, five hours and 67 moves later, both players were in shock. (Michal Walusza via FIDE)
The scenario could have been a lot different had the chips fallen differently in the Round 11 game against Vidit. Had the Indian managed to defeat Nepo, Gukesh and Nakamura would have been joint leaders. Had he managed to draw with the Russian, there would have been three joint leaders heading into Round 12.
When the end came, five hours and 67 moves later, both players were in shock. Vidit shook his head, held his face in both palms as he walked away. It was dawning on him that defeat left him two points behind the Russian, meaning his hopes of winning the Candidates are virtually over.
Nepo sat on his chair for a while, shuttered his eyes with his palm, scarcely believing that he had managed to claw a victory out of a game he seemed destined to lose. Twice.
How Vidit allowed Nepo to escape twice
By the 33rd move, Vidit, playing with white pieces, had built up a significant advantage on the board.
Chess Candidates
He could have added to that advantage by advancing his pawn in the ‘h file’ by one more square to h5, which would have put the black pawn on ‘g file’ under pressure: it could either capture the advancing pawn, or be captured by it. Either way, had Vidit made that move, there would be a pawn advancing towards Nepo’s king, not to mention the rook on the first rank who could have darted sideways before charging at Nepo’s king on h8.
Instead, Vidit chose the move 34. Kd3, which the computer didn’t appreciate too much.
“I was doing fine after that (move). But I wanted to win the game,” Nepo said before launching into an analysis of what he had thought during the game and its critical moments, a monologue he ended with: “I got lucky.”
He would get lucky again. The second reprieve Vidit inadvertently offered Nepomniachtchi was three moves later when instead of claiming the pawn on d5 with his knight, Vidit chose to withdraw his king to c2.
It was a decision that Vidit made in a hurry, because his time on the clock was thinning. Had Vidit chosen to claim the pawn with his knight, it would have started off a massacre on the board with two knights, two pawns, one bishop and two rooks losing their lives but Vidit emerging with a significant advantage.
Later, Nepo admitted he had not calculated what would happen if Vidit claimed the pawn with his knight. “Oh, white would win? Good to know,” he said with a straight face at the post-match press conference before the shadow of a smile appeared on his face as he added earnestly. “I didn’t calculate that of course.”
There was one more moment where Vidit could have chosen to repeat his move and push the game to a draw. But he chose differently.
On the 41st move, Vidit pushed his knight to d2, which got him a bishop d6 response. Vidit promptly shifted his knight to e4, attacking the black bishop. Nepo shifted it to the initial square again, which meant that the position had repeated twice on the board. All Vidit had to do was move his knight to d2 again. But he chose Ng3, which indicated he wasn’t looking for a draw.
“Luckily for me, he didn’t take the draw here and went Ng3,” Nepo said.
Nepomniachtchi has an enviable record at the Candidates. He won the 2020 edition, which was split in half by the coronavirus pandemic — the first half being played in March 2020 and the second half starting 13 months later in April 2021 in Yekaterinburg. But Nepo stayed consistent through that phase. He was joint leader when the event was stopped, and confirmed his victory with a round to spare when the players reconvened in 2021. In 2022, the event was clouded with a different kind of uncertainty: whether Magnus Carlsen will defend his title against the Candidates winner or not. Even at that edition, Nepo managed to win with a round to spare.
Chess CandidatesThe man who has a 100 percent success rate at the Candidates tournament is now one square ahead of the rest of the eight-player field. Gukesh and Nakamura have just three more moves left to catch the man who’s an escape artist on the 64 squares. (Michal Walusza via FIDE)
The man who has a 100 percent success rate at the Candidates tournament is now one square ahead of the rest of the eight-player field. Gukesh and Nakamura have just three more moves left to catch the man who’s an escape artist on the 64 squares.''')

In [ ]:
query = sentence + '\nTL;DR:'

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from datasets import Dataset, load_dataset

device =  "cuda"

model_ckpt = "facebook/bart-large-cnn"

tokenizer1 = AutoTokenizer.from_pretrained(model_ckpt)

model_summary = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device) 
pipe = pipeline('summarization', model=model_summary, tokenizer = tokenizer1,device = device)

In [ ]:
import torch
from transformers import pipeline

# Example function to generate segments of text within the maximum sequence length
def generate_segments(input_text):
    words = input_text.split()
    for i in range(0, len(words), 300):  # Adjust the segment length here
        yield ' '.join(words[i:i+300])   # Adjust the segment length here

def summarization(input_text):
    if(len(tokenizer.tokenize(input_text))<=1024):
        return input_text
    
    global count
    output_summary = ""
    for segment in generate_segments(input_text):
        try:
            max_length = len(segment.split()) # Adjust max_length dynamically
            summarized_segment = pipe(segment, max_length=max_length, min_length=1, do_sample=False)[0]['summary_text']
            output_summary += summarized_segment
            
            if len(output_summary.split()) >= 1024:
                break
        except IndexError:
            print("Index error occurred, skipping this segment.")
        except Exception as e:
            print(f"Error occurred: {e}")
    count += 1
    print(count, end=" ")
    return output_summary
count = 0

In [ ]:
%cd /kaggle/working

In [ ]:
model.save_pretrained("headline_GPT2")

In [2]:
from IPython.display import FileLink

file_path = "/kaggle/working/gpt2-fp16.tflite"  # Specify the full path to the file
link = FileLink(file_path)
link


/kaggle/working/gpt2-fp16.tflite

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Ateeqq/news-title-generator")
model = AutoModelForSeq2SeqLM.from_pretrained("Ateeqq/news-title-generator")


In [ ]:
def generate_title(input_text):

    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate the output
    output = model.generate(input_ids)

    # Decode the generated tokens
    decoded_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_text

# Example usage
input_text = df["article"][1900]
generated_title = generate_title(input_text)

# print(f"Input Text: {input_text}")
print(f"Generated Title: {generated_title}")


In [ ]:
df["title"][1900]

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from rouge import Rouge


# Define the generated summary and the reference summary
reference_summary = df["title"][1900]

# Initialize the ROUGE object
rouge = Rouge()


# Calculate ROUGE for the generated and reference summaries
scores = rouge.get_scores(generated_title, reference_summary)
# Print the results
print(scores)


In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
import transformers
from summarizer import Summarizer, TransformerSummarizer

In [ ]:
model_bert = TransformerSummarizer( transformer_model_key="mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization")

In [ ]:
model_bert = Summarizer()

In [ ]:
#helper  function
def summarize(model, text):
    summary = ''.join(model(text))
    return(summary)

In [ ]:
summarize(model_bert, df["article"][1900])

In [ ]:
df["article"][1900]

In [1]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel

model = TFGPT2LMHeadModel.from_pretrained('/kaggle/input/model-till-25000')

input_spec = tf.TensorSpec([1, 64], tf.int32)
model._set_inputs(input_spec, training=False)

print(model.inputs)
print(model.outputs)

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# FP16 quantization:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

open("gpt2-fp16.tflite", "wb").write(tflite_model)

2024-04-28 12:07:48.234891: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 12:07:48.235018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 12:07:48.369552: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


None
None


Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 319, Total Ops 3153, % non-converted = 10.12 %
 * 319 ARITH ops

- arith.constant:  319 occurrences  (f16: 299, i32: 20)



  (f32: 291, i32: 24)
  (f32: 49)
  (f32: 25)
  (f32: 299)
  (i32: 1)
  (f32: 96)
  (f32: 2)
  (f32: 24)
  (i1: 24)
  (f32: 98)
  (f32: 220)

  (f32: 24, i32: 220)
  (i32: 49)
  (f32: 313, i32: 27)
  (f32: 49)
  (i32: 219)
  (f32: 24)
  (f32: 24)
  (f32: 49)
  (i32: 461)
  (f32: 98, i32: 24)
  (f32: 96)


710222180